<a href="https://colab.research.google.com/github/ChhaviPanghal/InterviewInsights/blob/main/InterviewInsight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install -q kaggle

In [ ]:
# from google.colab import files

# files.upload()

# ! mkdir ~/.kaggle

# ! cp kaggle.json ~/.kaggle/

# ! chmod 600 ~/.kaggle/kaggle.json

# ! kaggle datasets download -d msambare/fer2013

# import zipfile
# zip_ref = zipfile.ZipFile('fer2013.zip', 'r')
# zip_ref.extractall('/content')
# zip_ref.close()

Saving kaggle.json to kaggle.json
100% 60.3M/60.3M [00:00<00:00, 222MB/s]
100% 60.3M/60.3M [00:00<00:00, 208MB/s]


In [ ]:
# from keras.models import Sequential
# from keras.layers import Conv2D,MaxPooling2D, Dense, Dropout, Flatten

In [ ]:
# from keras.optimizers import Adam
# from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# train_data_gen = ImageDataGenerator(rescale = 1/255)
# validation_data_gen = ImageDataGenerator(rescale = 1/255)

In [ ]:
# train_geneartor = train_data_gen.flow_from_directory("/content/train",target_size = (48,48),batch_size = 64,color_mode = "grayscale",class_mode = 'categorical')
# validation_data_gen = validation_data_gen.flow_from_directory("/content/test",target_size = (48,48),batch_size = 64,color_mode = "grayscale",class_mode = 'categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
# emotion_model = Sequential()
# emotion_model.add(Conv2D(32,kernel_size = (3,3),activation = 'relu',input_shape = (48,48,1)))
##emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
# emotion_model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu'))
# emotion_model.add(MaxPooling2D(pool_size = (2,2)))
# emotion_model.add(Dropout(0.25))

In [ ]:
# emotion_model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu'))
# emotion_model.add(MaxPooling2D(pool_size = (2,2)))
# emotion_model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu'))
# emotion_model.add(MaxPooling2D(pool_size = (2,2)))
# emotion_model.add(Dropout(0.25))

In [ ]:
# from keras.callbacks import EarlyStopping
# emotion_model.add(Flatten())
# emotion_model.add(Dense(1024,activation = 'relu'))
# emotion_model.add(Dropout(0.5))
# emotion_model.add(Dense(7,activation = 'softmax'))

# emotion_model.compile(loss = 'categorical_crossentropy',optimizer = Adam(lr = 0.0001),metrics = ['accuracy'])
# # early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')

In [ ]:
# emotion_model_info = emotion_model.fit_generator(train_geneartor,steps_per_epoch = 28709//64,epochs = 50,validation_data = validation_data_gen,validation_steps=7178//64)

In [ ]:
# model_json = emotion_model.to_json()

In [ ]:
# with open("emotion_model.json","w") as json_file:
#   json_file.write(model_json)

In [ ]:
# emotion_model.save_weights('emotion_model.h5')

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open("/content/emotion_model.json",'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

In [ ]:
emotion_model.load_weights("/content/emotion_model.h5")

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
face_cascade = cv2.CascadeClassifier("/content/haarcascade_frontalface_default.xml")

In [ ]:
eye_cascade = cv2.CascadeClassifier("/content/haarcascade_eye.xml")

In [ ]:
def detect_face(gray,frame):
  faces = face_cascade.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5,minSize = (200,200))
  for (x,y,w,h) in faces:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
  if len(faces)!=0:
    return faces[0]
  else:
    return (0,0,0,0)

In [ ]:
def detect_eyes(gray,frame):
  if frame is None:
    return 0
  faces = face_cascade.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5,minSize = (200,200))
  eyes_detected = []
  for (x,y,w,h) in faces:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    face_gray = gray[y:y+h,x:x+w]
    face_frame = frame[y:y+h,x:x+w]
    # print(cropped_img.shape)
    # show_image(cropped_img)
    eyes = eye_cascade.detectMultiScale(face_gray,scaleFactor = 1.2,minNeighbors = 6,minSize = (120,120))
    eye = (100000000,0,0,0)
    for (eye_x,eye_y,eye_w,eye_h) in eyes:
      if(eye_x<eye[0]):
        eye = (eye_x,eye_y,eye_w,eye_h)
        cv2.rectangle(face_frame,(eye[0],eye[1]),(eye[0]+eye[2],eye[1]+eye[3]),(0,255,0),2)
    try:
      eyes_detected.append((face_frame,eye))
    except:
      continue
  return eyes_detected

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def face_emotion_detection(video_path):
  cap = cv2.VideoCapture(video_path)
  diff_frames_emotions = []
  count = 0;
  while True:
    ret, frame = cap.read()
    try:
      frame = cv2.resize(frame,(1280,720))
    except:
      pass
    # show_image(frame)
    if not ret:
      print("Can't receive frame (stream end?). Exiting ...")
      break
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    (x,y,w,h)=detect_face(gray,frame)
    face_gray = gray[y:y+h,x:x+w]
    try:
       cropped_img = np.expand_dims(np.expand_dims(cv2.resize(face_gray,(48,48)),-1),0)
    except:
      continue
    # show_image(face_gray)
    diff_frames_emotions.append(int(np.argmax(emotion_model.predict(cropped_img))))
    if((int(np.argmax(emotion_model.predict(cropped_img))))==3 or (int(np.argmax(emotion_model.predict(cropped_img))))==4):
      count = count+1
  ans = 0
  ans = count/len(diff_frames_emotions)
  print(f"You appeared {ans*100}% if times neutral.")
  if(ans<=0.3):
    print("You can try to improve your facial expressions. Most of the times you doesn't seem to be happy or neutral.")
  if(ans<0.6 and ans>0.3):
    print("You can try improving your facial expressions a little.")

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
def eye_contact_detection(video_path):
  cap = cv2.VideoCapture(video_path)
  diff_frames_mean_eye = []
  while True:
    ret, frame = cap.read()
    if not ret:
      print("Can't receive frame (stream end). Exiting ...")
      break
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    # show_image(gray)
    for image,coordinates in detect_eyes(gray,frame):
      eye_image = image[coordinates[1]:coordinates[1]+coordinates[3],coordinates[0]:coordinates[0]+coordinates[2]]
      try:
        gray_eye_image = cv2.cvtColor(eye_image,cv2.COLOR_BGR2GRAY)
      except:
        continue
      _,threshold = cv2.threshold(gray_eye_image,33,220,cv2.THRESH_BINARY_INV)
      count = []
      for i in range(len(threshold)):
        for j in range(len(threshold)):
          if threshold[i][j]==220:
            count.append((i,j))
      sum_x = 0
      mean_x = 0
      sum_y = 0
      mean_y = 0
      for i in count:
        sum_x+=i[0]
        sum_y+=i[1]
      try:
        mean_x = sum_x/len(count)
        mean_y = sum_y/len(count)
      except:
        None
      diff_frames_mean_eye.append((mean_x,mean_y))
  diff_frames_mean_eye = np.array(diff_frames_mean_eye)
  mean_eyes = np.mean(diff_frames_mean_eye)
  std_dev = np.std(diff_frames_mean_eye)
  diff_frames_mean_eye = (diff_frames_mean_eye - mean_eyes)/std_dev
  out = 0;
  for a in diff_frames_mean_eye:
    if((a[0]<= -1.3 or a[0]>=1.3) or (a[1]<= -1.3 or a[1]>=1.3)):
      out = out+1
  eye_contact =  out/len(diff_frames_mean_eye)
  if(eye_contact<0.2):
    print("Heyyy! You have a perfect eye conatct")
  if(eye_contact>0.15 and eye_contact<0.3):
    print("A very little improvement required in eye Contact :-(. You can use the Triangle Technique i.e.focus on the area between the person's eyes or the bridge of their nose. This gives the appearance of eye contact without feeling as intense as staring directly into someone's eyes.")
  if(eye_contact>0.3):
    print("A lot of improvement in your eye contact is required. You should try to use Triangle techniques and also practicing regularly.")

In [ ]:
eye_contact_detection("/content/pexels-tima-miroshnichenko-5439086 (2160p).mp4")

Can't receive frame (stream end). Exiting ...
Heyyy! You have a perfect eye conatct


In [46]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 26.8 MB/s eta 0:00:00


In [47]:
import speech_recognition as sr

In [48]:
r = sr.Recognizer()

In [49]:
def audio_to_text(audio_path):
  with sr.AudioFile(audio_path) as source:
    audio_text = r.listen(source)
  try:
    text = r.recognize_google(audio_text)
    print('Converting audio transcripts into text ...')
    # print(text)
  except:
    print('Sorry.. run again...')
  return text

In [1]:
import pandas as pd
essay = pd.read_csv("/content/essaytrain.csv",encoding = 'ISO-8859-1')

In [2]:
essay.head()

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,2003_199.txt,"well, here I go writing a stream of consciousn...",y,n,y,n,y
1,1997_894347.txt,as I begin this assignment its seems weird to ...,y,n,n,y,n
2,1999_559746.txt,I can't make up my mind if I want to go to a m...,y,n,y,y,n
3,1999_794811.txt,why am I so stressed out is it school or is it...,n,y,n,n,n
4,2004_327.txt,I'm thinking about if I am going to get the jo...,y,y,n,y,y


In [3]:
trait_columns = ['cOPN', 'cCON', 'cEXT', 'cAGR', 'cNEU']
d = {'y': 1, 'n': 0}
for trait in trait_columns:
  essay[trait] = essay[trait].map(d)

In [4]:
essay.head()

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,2003_199.txt,"well, here I go writing a stream of consciousn...",1,0,1,0,1
1,1997_894347.txt,as I begin this assignment its seems weird to ...,1,0,0,1,0
2,1999_559746.txt,I can't make up my mind if I want to go to a m...,1,0,1,1,0
3,1999_794811.txt,why am I so stressed out is it school or is it...,0,1,0,0,0
4,2004_327.txt,I'm thinking about if I am going to get the jo...,1,1,0,1,1


In [71]:
X = essay['TEXT']

In [72]:
y = essay[trait_columns]

In [73]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [74]:
import re
import string

stop_words = stopwords.words("english")

def clean_text(text):
    text = text.lower()
    # remove all numbers
    text = re.sub("\d", " ", text)
    # remove punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    # remove extra spaces
    text = re.sub('\n', ' ', text)
    text = re.sub('\s{2,}',' ', text)
    # stopwords
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    # remove non-Latin characters
    text = re.sub('[^a-zA-Z\s]', '', text)
    text = word_tokenize(text)
    list_add = []
    for word in text:
      list_add.append(lemmatizer.lemmatize(word,'v'))
    text = ' '.join(list_add)
    return text

In [75]:
X= X.apply(clean_text)

In [57]:
max_len = 0;

In [76]:
list_add = []
for i in range(X.shape[0]):
  X[i] = word_tokenize(X[i])
  list_add = []
  for word in X[i]:
    list_add.append(lemmatizer.lemmatize(word,'v'))
  if(len(list_add)>max_len):
    max_len = len(list_add)
  X[i] = ' '.join(list_add)

In [77]:
X

0       well go write stream consciousness starve righ...
1       begin assignment seem weird college locate pro...
2       make mind want go meet charles ask go kind wan...
3       stress school go life clue probably becasue fr...
4       think go get job want pac university would suc...
                              ...                        
1475    hear roommate click mouse search something int...
1476    arrive home go home weekend labor day spend ti...
1477    think whole thing ridiculous question obvious ...
1478    right feel tire usually look key board notice ...
1479    almost do pre test survey think do would finis...
Name: TEXT, Length: 1480, dtype: object

In [60]:
# max_len

In [61]:
# from numpy import array
# from numpy import asarray
# from numpy import zeros
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers import Embedding

In [62]:
# t = Tokenizer()
# t.fit_on_texts(X)
# vocab_size = len(t.word_index) + 1
# encoded_docs = t.texts_to_sequences(X)
# print(encoded_docs)

In [63]:
# padded_docs = pad_sequences(encoded_docs, maxlen=max_len, padding='post')
# print(padded_docs)

In [64]:

# embeddings_index = dict()
# f = open('/content/glove.6B.100d.txt')
# for line in f:
# 	values = line.split()
# 	word = values[0]
# 	coefs = asarray(values[1:], dtype='float32')
# 	embeddings_index[word] = coefs
# f.close()
# print('Loaded %s word vectors.' % len(embeddings_index))

In [65]:
# embedding_matrix = zeros((vocab_size, 100))
# for word, i in t.word_index.items():
# 	embedding_vector = embeddings_index.get(word)
# 	if embedding_vector is not None:
# 		embedding_matrix[i] = embedding_vector

In [66]:
# # define model
# model = Sequential()
# e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)
# model.add(e)
# model.add(Flatten())
# model.add(Dense(100,activation = 'relu'))
# model.add(Dense(5, activation='sigmoid'))
# # compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# # summarize the model
# print(model.summary())

In [67]:
# # fit the model
# history = model.fit(padded_docs,y, epochs=25, verbose=0)
# # evaluate the model
# loss, accuracy = model.evaluate(padded_docs, y, verbose=0)
# print('Accuracy: %f' % (accuracy*100))

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X=tfidf.fit_transform(X)
def tf_idf_vectorize(text):
  text = [text]
  return tfidf.transform(text)

In [ ]:
# X_transformed

# X_train, X_test, y_train, y_test = train_test_split(X_transformed,y, test_size=0.2)

In [ ]:
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import ConfusionMatrixDisplay
# from sklearn.metrics import accuracy_score, classification_report


# def eval_model(y_true,y_predicted):
#     cm = confusion_matrix(y_true, y_predicted)
#     cm_display=ConfusionMatrixDisplay(confusion_matrix = cm)
#     print('accuracy score',accuracy_score(y_true, y_predicted))
#     print(classification_report(y_true,y_predicted))

In [79]:
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
# for i in ["cOPN", "cCON", "cEXT", "cAGR", "cNEU"]:
#   LRparam_grid = {
#     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
#     'penalty': ['l1', 'l2'],
#     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
#   grid = GridSearchCV(LogisticRegression(), LRparam_grid, refit = True, verbose = 3)
#   grid.fit(X_train, y_train[i])
#   print(grid.best_params_)
#   eval_model(y_test[i],grid.predict(X_test))


#out of three classifier for cEXT, cAGR, cNEU Logistic Regression is best..

In [80]:
from sklearn.svm import SVC
# for i in ["cOPN", "cCON", "cEXT", "cAGR", "cNEU"]:
#   param_grid = {'C': [0.1, 1, 10, 100, 1000],'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']}
#   grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
#   grid.fit(X_train, y_train[i])
#   print(grid.best_params_)
#   eval_model(y_test[i],grid.predict(X_test))

#out of three classifier for cCON Logistic Regression is best..

In [81]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# for i in ["cOPN", "cCON", "cEXT", "cAGR", "cNEU"]:
#   param_grid = {'n_estimators': [25, 50, 100, 150],'max_features': ['sqrt', 'log2', None],'max_depth': [3, 6, 9],'max_leaf_nodes': [3, 6, 9]}
#   grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3)
#   grid.fit(X_train, y_train[i])
#   print(grid.best_params_)
#   eval_model(y_test[i],grid.predict(X_test))

#out of three classifier for cOPN Logistic Regression is best..

In [82]:
model_opn = RandomForestClassifier(n_estimators = 9,max_leaf_nodes = 9,max_features=None,max_depth = 9)
model_opn.fit(X,y["cOPN"])

RandomForestClassifier(max_depth=9, max_features=None, max_leaf_nodes=9,
                       n_estimators=9)

In [83]:
model_con = SVC(C=10,gamma = 1,kernel = 'rbf')
model_con.fit(X,y["cCON"])

SVC(C=10, gamma=1)

In [84]:
model_ext = LogisticRegression(C=1,penalty = 'l2',solver = 'saga')
model_ext.fit(X,y["cEXT"])

LogisticRegression(C=1, solver='saga')

In [85]:
model_agr = LogisticRegression(C=1,penalty = 'l2',solver = 'newton-cg')
model_agr.fit(X,y["cAGR"])

LogisticRegression(C=1, solver='newton-cg')

In [86]:
model_neu = LogisticRegression(C=1000,penalty = 'l2',solver = 'sag')
model_neu.fit(X,y["cNEU"])

  warnings.warn(



LogisticRegression(C=1000, solver='sag')

In [87]:
def predict_personality(text):
  text = tf_idf_vectorize(text)
  #for cOPN
  if(model_opn.predict(text)):
    print("You seemed to be organized, detail-oriented, and responsible.Also it appears that you are punctual, reliable, and committed to their work.".upper())
  if(not model_opn.predict(text)):
    print("Seems like you are less-responsible and less detail-oriented. Also it appears that you are not much committed to your work.")
  if(model_con.predict(text)):
    print("You tend to be creative, adaptable, and curious. You might bring fresh perspectives and innovative ideas to the workplace.")
  if(not model_con.predict(text)):
    print("Your style of narration seems to be casual. It doesn't signifies the creativity within you.")
  if(model_ext.predict(text)):
    print("You are seen as sociable, energetic, and good communicators. You may excel in roles that involve teamwork, networking, or client interactions.")
  if(not model_con.predict(text)):
    print("You don't seem to be energetic and not so good communicator. Also you seem to lack teamwork")
  if(model_agr.predict(text)):
    print("You seems to be cooperative, empathetic, and good team players. You tend to build positive relationships and contribute to a harmonious work environment.")
  if(not model_agr.predict(text)):
    print("You don't seems to be cooperative, empathetic.")
  if(model_neu.predict(text)):
    print("You seems to be anxious or emotionally reactive:(")
  if(not model_neu.predict(text)):
    print("You seems to be calm, resilient, and emotionally stable, making you reliable in high-pressure situations :)")

In [88]:
!pip install librosa

In [89]:
!pip install entropy

In [33]:
!pip install praat-parselmouth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 17.9 MB/s eta 0:00:00


In [34]:
import entropy as ent

In [35]:
import parselmouth

In [36]:
import moviepy.editor as mp

In [37]:
import librosa
import numpy as np

In [38]:
def pitch_find(signal):
  snd = parselmouth.Sound(signal)
  pitch = snd.to_pitch()
  pitch_values = pitch.selected_array['frequency']
  return pitch_values

In [39]:
def confidence(signal,audio,sr):
  feature_weights = {
        "zero_crossing_rate": 0.14,
        "spectral_centroid": 0.14,
        "pitch": 0.108,
        "mfccs": 0.14,
    }
  zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(audio))
  spectral_centroid = np.mean( librosa.feature.spectral_centroid(y=audio, sr=sr))
  pitch = np.mean(pitch_find(signal))
  MFCC = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13))

  boldness_metric = (
        feature_weights["zero_crossing_rate"] * zero_crossing_rate +
        feature_weights["spectral_centroid"] * spectral_centroid +
        feature_weights["mfccs"] * MFCC+
        feature_weights["pitch"]*pitch
    )

  max_possible_score = sum(feature_weights.values())
  boldness_score = (boldness_metric / max_possible_score) *0.1
  print(f"Your bolness score is {boldness_score}")
  if(boldness_score>=80):
    print("Bold and Clear voice")
  elif(boldness_score>=60):
    print("Pretty good bold and clear voice but can improve")
  else:
    print("You need to work hard on your voice. Boldness and clarity lacks in your voice.")
  return boldness_score

In [40]:
from moviepy.editor import *
def audio_file(video_path):
  video = VideoFileClip(video_path)
  audio = video.audio
  audio.write_audiofile("/content/audio_file.wav")
  return audio

In [41]:
def interviewInsight(video_path):
  face_emotion_detection(video_path)
  eye_contact_detection(video_path)
  try:
    audio = audio_file(video_path)
    text = audio_to_text("/content/audio_file.wav")
    text=clean_text(text)
    predict_personality(text)
    y,sr = librosa.load("/content/audio_file.wav")
    confidence("/content/audio_file.wav",y,sr)
  except:
    pass

In [44]:
def get_audio_insight(audio):
  text = audio_to_text(audio)
  text=clean_text(text)
  predict_personality(text)
  y,sr = librosa.load(audio)
  confidence(audio,y,sr)

In [90]:
get_audio_insight("/content/ChangeTheWorld.wav")

Converting audio transcripts into text ...
hi this is AJ today I'm going to talk to you about our mission because you and I are going to change the world I'm very excited about this there is a problem in the world I have traveled to many countries around the world and everywhere I find the same problem
You seemed to be organized, detail-oriented, and responsible.Also it appears that you are punctual, reliable, and committed to their work.
Your style of narration seems to be casual. It doesn't signifies the creativity within you.
You are seen as sociable, energetic, and good communicators. You may excel in roles that involve teamwork, networking, or client interactions.
You don't seem to be less energetic and not so good communicator. Also you seem to lack teamwork
You don't seems to be cooperative, empathetic.
You seems to be calm, resilient, and emotionally stable, making you reliable in high-pressure situations :)


Missing samples were set to zero.
  snd = parselmouth.Sound(signal)



Your bolness score is 77.43418879070063
Pretty good bold and clear voice but can improve
